In [59]:
#loading data
import pandas as pd
import numpy as np
import tensorflow as tf
train = pd.read_csv('train.csv')
test= pd.read_csv('test.csv')

train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Id
count,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000,466.000000
mean,4.032189,122.096567,68.341202,20.946352,82.319742,32.504721,0.474120,33.206009,0.377682,377.030043
std,3.395290,32.507688,20.970781,16.295765,114.428297,7.757115,0.322178,11.451461,0.485329,222.736855
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000,0.000000
25%,1.000000,100.000000,62.000000,0.000000,0.000000,27.600000,0.240250,24.000000,0.000000,185.500000
50%,3.000000,119.000000,70.000000,23.000000,39.000000,32.450000,0.386500,29.000000,0.000000,372.500000
75%,7.000000,142.000000,80.000000,33.000000,140.000000,37.400000,0.635500,41.000000,1.000000,576.750000
max,15.000000,199.000000,122.000000,99.000000,744.000000,67.100000,2.329000,81.000000,1.000000,766.000000


In [60]:
#imputing zero values with mean/2
columns = ['Glucose', 'BloodPressure','Insulin','SkinThickness','BMI','Age']
for fields in columns:
    col_train=train[fields]    
    col_train=col_train.replace(0,col_train.mean()/2)
    train[fields]=col_train
    col_test=test[fields]
    col_test=col_test.replace(0,col_test.mean()/2)
    test[fields]=col_test
    
train.head(10)    

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Id
0,6,148.0,72.000000,35.000000,41.159871,33.6,0.627,50,1,0
1,8,183.0,64.000000,10.473176,41.159871,23.3,0.672,32,1,2
2,1,89.0,66.000000,23.000000,94.000000,28.1,0.167,21,0,3
3,10,115.0,34.170601,10.473176,41.159871,35.3,0.134,29,0,7
4,2,197.0,70.000000,45.000000,543.000000,30.5,0.158,53,1,8
5,4,110.0,92.000000,10.473176,41.159871,37.6,0.191,30,0,10
6,10,168.0,74.000000,10.473176,41.159871,38.0,0.537,34,1,11
7,5,166.0,72.000000,19.000000,175.000000,25.8,0.587,51,1,14
8,7,100.0,34.170601,10.473176,41.159871,30.0,0.484,32,1,15
9,0,118.0,84.000000,47.000000,230.000000,45.8,0.551,31,1,16


In [61]:
from sklearn.preprocessing import MinMaxScaler
columns_scale = ['Pregnancies', 'BloodPressure','Insulin','SkinThickness','BMI','Age']
for scle in columns_scale:
    scaler =MinMaxScaler()
    scale_train = pd.DataFrame(train, columns= [scle])
    scaled_train = scaler.fit_transform(scale_train)
    train[scle] = scaled_train
    
    scale_test = pd.DataFrame(test, columns = [scle])
    scaled_test = scaler.fit_transform(scale_test)
    test[scle] = scaled_test
  
"""  
train['DiabetesPedigreeFunction'] =train['DiabetesPedigreeFunction'] *10
test['DiabetesPedigreeFunction'] =test['DiabetesPedigreeFunction'] *10
"""

train.drop(['Id'], axis = 1, inplace =True)
test.drop(['Id'], axis = 1, inplace =True)

Y_train = train['Outcome']
X_train = train.drop(['Outcome'], axis=1)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.2)


In [68]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()

n_estimators = [int(x) for x in np.linspace(5,500,50)]
max_depth = [int(x) for x in np.linspace(5,50,36)]
min_samples_split = [int(x) for x in np.linspace(1,20,20)]
criterion = ['entropy','gini']
min_samples_leaf = [int(x) for x in np.linspace(1,20,15)] 
hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split
             , criterion=criterion,min_samples_leaf=min_samples_leaf)

from sklearn.model_selection import RandomizedSearchCV
gridF = RandomizedSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train,Y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    6.0s finished
C:\Users\asus\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [74]:
bestF.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=47, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=287,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [80]:
forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=47, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=287,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


0.9721030042918455

In [ ]:
Not overfitting

In [94]:
forest.score(x_train,y_train)
y_pred = forest.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
acc

0.9574468085106383

In [95]:
forest.fit(X_train,Y_train)
y_pred = forest.predict(test)

In [ ]:
y_pred = pd.DataFrame(y_pred)
sub = pd.read_csv('sample_submission.csv')
dataset = pd.concat([sub['Id'],y_pred],axis =1)
dataset.columns=['Id','Outcome']
dataset.to_csv('soln.csv', index =False)